In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import tfscreen

from tfscreen import build_condition_dataframes
from tfscreen import generate_libraries
from tfscreen import generate_phenotypes
from tfscreen import transform_and_mix
from tfscreen import initialize_population
from tfscreen import simulate_growth
from tfscreen import sequence_conditions

import random
import copy


In [2]:
# -----------------------------------------------------------------------------
# Ensemble information

# spreadsheet defining ensemble and interactions with iptg
ensemble_spreadsheet = "input/ensemble.xlsx"

# Spreadsheet with affects of all mutations on each state in ensemble
ddG_spreadsheet = "input/ddG.xlsx"

T = 310      # K
R = 0.001987 # kcal/mol/K

# This sets how the thermodynamic variable maps proportionally to output. For
# the lac repressor, this is 12 because we dervied the thermodynamics 
# parameters under conditions where there was a 12-fold excess of the lac 
# repressor to operator. Fully bound lac repressor thus corresponds to a 
# an observable value of 1/12 in the operator-bound state--hence 12 here.  
scale_obs_by = 12

# -----------------------------------------------------------------------------
# Library definition information

# wildtype amino acid sequence (single letter, uppercase)
aa_sequence = \
"""
VKPVTLYDVAEYAGVSYQTVSRVVNQASHVSAKTREKVEAAMAELNYIPNRVAQQLAGK
QSLLIGVATSSLALHAPSQIVAAIKSRADQLGASVVVSMVERSGVEACKAAVHNLLAQR
VSGLIINYPLDDQDAIAVEAACTNVPALFLDVSDQTPINSIIFSHEDGTRLGVEHLVAL
GHQQIALLAGPLSSVSARLRLAGWHKYLTRNQIQPIAEREGDWSAMSGFQQTMQMLNEG
IVPTAMLVANDQMALGAMRAITESGLRVGADISVVGYDDTEDSSCYIPPLTTIKQDFRL
LGQTSVDRLLQLSQGQAVKGNQLLPVSLVKRKTTLAPNTQTASP
"""

# sites that are mutated. sites should either be wildtype 
# amino acids or non-uppercase letters. non-uppercase letters
# are interpreted as sub-library identifiers. 
mutated_sites = \
"""
VKPVTLYDVAEYAGVSYQTVSRVVNQASH111111111111111111111111111111
QS2222222222222222222222222222222222222VERSGVEACKAAVHNLLAQR
VSGLIINYPLDDQDAIAVEAACTNVPALFLDVSDQTPINSIIFSHEDGTRLGVEHLVAL
GHQQIALLAGPLSSVSARLRLAGWHKYLTRNQIQPIAEREGDWSAMSGFQQTMQMLNEG
IVPTAMLVANDQMALGAMRAITESGLRVGADISVVGYDDTEDSSCYIPPLTTIKQDFRL
LGQTSVDRLLQLSQGQAVKGNQLLPVSLVKRKTTLAPNTQTASP
"""

# Number of first letter in amino sequence. Should match 
# numbering in ddG spreadsheet
seq_starts_at = 1

# Should we make internal combinations (within O1, O2, etc.)
internal_doubles = False 

# highest order combination to make (1 -- alone, 2 -- pairs,
# 3 -- triplicates, etc.)
max_num_combos = 2

# What codon to place at mutated sites
degen_codon = "nnt"

# -----------------------------------------------------------------------------
# Transformation and library composition

# Define how many transformants to include for each sub-library. Keys are 
# tuples of strings. ('1',) means O1; ('2',) means O2; ('1','2') means O1/O2. 
# if you set internal_doubles to True above, you can then add ('1','1') and 
# ('2','2') to get the doubles within each pool. 

# sizes: how many clones will you get for the smallest transformation of this
# library
transform_sizes = {('1',):1e5,
                   ('2',):1e5,
                   ('1','2'):1e6}

# mixture: what is the mixture ratio of the library. This works together with 
# sizes. For the example values, the final mixed library would have 1e5 * 1
# O1, 1e5 * 1, O2, and and 1e6 * 10 O1/O2. Note that the **ratio** matters, 
# not the absolute value. You do not need to have all 1E10 clones... in fact,
# keep as small as possible while maintinaing ratios. I'd recommend setting 
# the lowest count library bit to 1, then scaling rest appropriately.
library_mixture = {('1',):1,
                   ('2',):1,
                   ('1','2'):10}

# The number of plasmids per cell is set by sampling from a Poisson 
# distribuation. If 0 or None, each cell gets exactly one plasmid. Otherwise,
# do Poisson sampling.
lambda_value = 3

# Maximum number of plasmids per cell. The lower this number, the faster the
# analysis; however, if you make it too small relative to lambda_value, you'll
# distort the Poisson distribution by cutting off the long right tail. 10 should
# be safe for lambda ~ 2-3. 
max_num_plasmids = 10 

# Standard deviation on the base growth rate vis-a-vis wildtype. 
mut_growth_rate_std = 0.01


# -----------------------------------------------------------------------------
# Pre selection culture set up

# Number of cells that come out of glycerol stock
# (NIH says 40%)
num_thawed_colonies = 1e7

# Overnight cell culture volume
overnight_volume_in_mL = 10

# Grow cells to this cfu/mL before diluting into IPTG 
# (OD600 of 0.35 on spec is 89370229 cfu/mL)
pre_iptg_cfu_mL = 90000000

# Grow out this long before putting into selection conditions
iptg_out_growth_time = 30

# Dilute by this factor when adding to IPTG tubes
post_iptg_dilution_factor = 0.2/10.2



# -----------------------------------------------------------------------------
# Selection conditions

# condition_blocks is a list of dictionaries defining different growt
# conditions. Each dictionary should have a "marker" key with a value of either
# "kanR" or "pheS", a "select" key with a value of either 0 or 1, an "iptg"
# key with a list of iptg concentrations (in mM), and a "time" key with a list
# of sample times (in minutes)

condition_blocks = [{"iptg":[0,1],
                     "marker":"kanR",
                     "select":0,
                     "time":[80,95,110]},
                    {"iptg":[0,1],
                     "marker":"pheS",
                     "select":0,
                     "time":[80,95,110]},
                    {"iptg":[0,0.0001,0.001,0.003],
                     "marker":"kanR",
                     "select":1,
                     "time":[160,180,200]},
                    {"iptg":[0.01,0.03,0.1,1.0],
                     "marker":"kanR",
                     "select":1,
                     "time":[145,165,180]},
                    {"iptg":[0, 0.0001, 0.001, 0.003, 0.01, 0.03, 0.1, 1.0],
                     "marker":"pheS",
                     "select":1,
                     "time":[95,110,125]}]
                     
# -----------------------------------------------------------------------------
# Data collection parameters

total_num_reads = 1e9

# This populates the replicate column in the output dataframes, exactly as 
# happens experimentally. Not really used, but could be helpful if you wanted 
# to combine multiple sims. 
replicate = 1



In [3]:
# -----------------------------------------------------------------------------
# Build library

libraries, genotype_df = generate_libraries(aa_sequence=aa_sequence,
                                            mutated_sites=mutated_sites,
                                            seq_starts_at=seq_starts_at,
                                            max_num_combos=max_num_combos,
                                            internal_doubles=internal_doubles,
                                            degen_codon=degen_codon)

condition_df, condition_df_with_time = build_condition_dataframes(condition_blocks,
                                                                  replicate=replicate)

# -----------------------------------------------------------------------------
# Calculate phenotypes

# Calculate phenotype. Each row is indexed by a unique genotype
phenotype_df, genotype_df = generate_phenotypes(genotype_df=genotype_df,
                                                condition_df=condition_df,
                                                ensemble_spreadsheet=ensemble_spreadsheet,
                                                ddG_spreadsheet=ddG_spreadsheet,
                                                scale_obs_by=scale_obs_by,
                                                mut_growth_rate_std=mut_growth_rate_std,
                                                T=T,
                                                R=R)

# -----------------------------------------------------------------------------
# Sample from library and grow out

# Sample from the main library
input_library = transform_and_mix(libraries=libraries,
                                  transform_sizes=transform_sizes,
                                  library_mixture=library_mixture,
                                  max_num_plasmids=max_num_plasmids,
                                  lambda_value=lambda_value)


# Create initial populations and growth rates for each cell under all relevant 
# conditions
init_output = initialize_population(input_library=input_library,
                                    phenotype_df=phenotype_df,
                                    genotype_df=genotype_df,
                                    condition_df=condition_df,
                                    num_thawed_colonies=num_thawed_colonies,
                                    overnight_volume_in_mL=overnight_volume_in_mL,
                                    pre_iptg_cfu_mL=pre_iptg_cfu_mL,
                                    iptg_out_growth_time=iptg_out_growth_time,
                                    post_iptg_dilution_factor=post_iptg_dilution_factor)
    
bacteria, ln_pop_array, bact_condition_k = init_output

# Simulate the growth if of each condition. This creates a dictionary keying 
# conditions to ln_pop vectors for each bacterium. condition_df_with_time
# gets updated to include the total population at each 
condition_pops = simulate_growth(ln_pop_array,
                                 bact_condition_k,
                                 condition_df,                                          
                                 condition_df_with_time)

count_df, condition_df_with_time = sequence_conditions(condition_pops,
                                                       condition_df_with_time,
                                                       genotype_df,
                                                       bacteria,
                                                       total_num_reads)


calculating phenotypes


  0%|          | 0/224890 [00:00<?, ?it/s]

storing phenotypes
Generating initial populations


  0%|          | 0/10200000 [00:00<?, ?it/s]

Getting bacterial growth rates


...

Growing populations in specified conditions


  0%|          | 0/20 [00:00<?, ?it/s]

Sequencing final populations


  0%|          | 0/60 [00:00<?, ?it/s]

Building final dataframe


In [12]:
# counter = 0
# base_condition_number = {}
# for c in condition_df_with_time.index:
#     base_condition = "-".join(c.split("-")[:-1])
#     if base_condition not in base_condition_number:
#         base_condition_number[base_condition] = counter
#         counter += 1
        
# base_condition_number
#count_df.loc[count_df["genotype"] == "wt",:] #.sort_values(["genotype_number","base_condition_number","time"])
#count_df.index = np.arange(len(count_df["genotype"]),dtype=int)
count_df.loc[count_df["counts"] == 201830,:]
count_df.loc[count_df["genotype"] == "V30L/A92Y",:]

,genotype,genotype_number,condition,base_condition,base_condition_number,time,counts
genotype,,,,,,,
V30L/A92Y,V30L/A92Y,6985,1-kanR-0-0.0-80,1-kanR-0-0.0,0,80,1
V30L/A92Y,V30L/A92Y,6985,1-kanR-0-0.0-95,1-kanR-0-0.0,0,95,1
V30L/A92Y,V30L/A92Y,6985,1-kanR-0-0.0-110,1-kanR-0-0.0,0,110,1
V30L/A92Y,V30L/A92Y,6985,1-kanR-0-1.0-80,1-kanR-0-1.0,1,80,12
V30L/A92Y,V30L/A92Y,6985,1-kanR-0-1.0-95,1-kanR-0-1.0,1,95,7
V30L/A92Y,V30L/A92Y,6985,1-kanR-0-1.0-110,1-kanR-0-1.0,1,110,9
V30L/A92Y,V30L/A92Y,6985,1-kanR-1-0.0-160,1-kanR-1-0.0,2,160,0
V30L/A92Y,V30L/A92Y,6985,1-kanR-1-0.0-180,1-kanR-1-0.0,2,180,0
V30L/A92Y,V30L/A92Y,6985,1-kanR-1-0.0-200,1-kanR-1-0.0,2,200,0


In [ ]:
g

In [ ]:
ct = condition_df_with_time.copy()
ct["cfu_per_mL"] = 0.0

all_pops = {}
for i in range(ln_pop_array.shape[1]):
    
    c = condition_df.loc[condition_df.index[i],["replicate",
                                                "marker",
                                                "select",
                                                "iptg"]]
    mask = (ct["replicate"] == c["replicate"]) & \
           (ct["marker"] == c["marker"]) & \
           (ct["select"] == c["select"]) & \
           (ct["iptg"] == c["iptg"])

    delta_time = np.array(ct.loc[mask,"time"],dtype=float)
    delta_time[1:] = delta_time[1:] - delta_time[:-1]

    condition_ids = ct.index[mask]

    p = None
    for j in range(len(condition_ids)):
        
        if p is None:
            p = ln_pop_array[:,i]
        p = tfscreen.grow_for_time(ln_pop_array=p,
                                   growth_rates=bact_condition_k[:,i],
                                   t=dt)
        all_pops[condition_ids[j]] = p

        ct.loc[condition_ids[j],"cfu_per_mL"] = np.sum(np.exp(p))

num_samples = len(ct)
ct["num_reads"] = int(total_num_reads//num_samples)
bacteria_as_int = np.arange(len(bacteria),dtype=int)

sequencing_dataframes = []
for condition in tqdm(all_pops):

    num_reads = ct.loc[condition,"num_reads"]
    
    pop = all_pops[condition]
    w = np.exp(pop)
    w = w/np.sum(w)
    sample = np.random.choice(bacteria_as_int,size=num_reads,p=w)

    this_df = genotype_df.copy()
    this_df.insert(loc=0,column="condition",value=condition)
    this_df["counts"] = 0

    seen, counts = np.unique(sample,return_counts=True)
    bacteria_seen = bacteria[seen]

    if not issubclass(type(bacteria_seen[0]),str):
        
        new_bacteria_seen = []
        new_counts = []
        
        for i in range(len(bacteria_seen)):
            genotypes = bacteria_seen[0]
            sub_counts = counts[0]
            plasmids_pulled = np.random.choice(genotypes,size=sub_counts)
            this_g, this_count = np.unique(plasmids_pulled,return_counts=True)
            new_bacteria_seen.extend(this_g)
            new_counts.extend(this_count)

        bacteria_seen = new_bacteria_seen
        counts = new_counts
    
    this_df.loc[bacteria_seen,"counts"] = counts

    sequencing_dataframes.append(this_df)
    
count_df = pd.concat(sequencing_dataframes)

In [ ]:
count_df[count_df["counts"] > 0]

In [ ]:


# Get the population over time for each each selection and iptg condition
pops_vs_time = growth_with_selection(ln_pop_array,
                                     bact_condition_k,
                                     time_points=sample_times)

In [ ]:


# -----------------------------------------------------------------------------
# Sequence samples and summarize results
df, genotype_df, condition_df = sequence_and_collate(pops_vs_time=pops_vs_time,
                                                     iptg_concs=iptg_concs,
                                                     sample_times=sample_times,
                                                     input_library=input_library,
                                                     all_genotypes=all_genotypes,
                                                     num_reads_per_condition=num_reads_per_condition)



In [ ]:
ct

In [ ]:

print("Generating initial populations",flush=True)
if input_library.ndim == 1:

    bacteria, ln_pop_array = tfscreen.thaw_glycerol_stock(input_library)
    bacteria, counts = np.unique(bacteria,return_counts=True)
    ln_pop_array = np.log(np.exp(ln_pop_array[0])*counts)

else:

    to_thaw = []
    indexer = {}
    for i in tqdm(range(input_library.shape[0])):
        
        tmp_genos = list(input_library[i][input_library[i] != "-"])
        tmp_genos.sort()
        tmp_genos = tuple(tmp_genos)
        
        if tmp_genos not in indexer:
            indexer[tmp_genos] = i
        
        to_thaw.append(indexer[tmp_genos])

    to_thaw = np.array(to_thaw)
    reverse_indexer = dict(zip(indexer.values(),indexer.keys()))

    bacteria, ln_pop_array = tfscreen.thaw_glycerol_stock(to_thaw)
    bacteria, counts = np.unique(bacteria,return_counts=True)
    ln_pop_array = np.log(np.exp(ln_pop_array[0])*counts)

    bacteria = np.array([reverse_indexer[b] for b in bacteria],dtype=object)
    

In [ ]:
plt.hist(ln_pop_array,bins=np.arange(-6,6,0.1))


In [ ]:
bacteria, population = tfscreen.thaw_glycerol_stock(out)
bacteria, counts = np.unique(bacteria,return_counts=True)
population = np.log(np.exp(population[0])*counts)







genotype_to_idx = dict(zip(list(genotype_df.index),
                           range(len(genotype_df.index))))

num_conditions = condition_df.shape[0]
num_bacteria = len(input_library)

condition_k = np.array(phenotype_df["growth_rate"])
condition_k = np.reshape(condition_k,
                         (len(condition_k)//num_conditions,
                          num_conditions))
base_delta_k = np.array(genotype_df["growth_rate_effect"])


if input_library.ndim == 1:
    
    idx = np.array([genotype_to_idx[g] for g in input_library])

    clone_genotypes = input_library
    clone_base_delta_k = base_delta_k[idx]
    clone_condition_k = condition_k[idx]

else:

    clone_genotypes = []
    clone_condition_k = np.zeros((num_bacteria,num_conditions),dtype=float)
    clone_base_delta_k = np.zeros(num_bacteria)
    
    for i in tqdm(range(len(input_library))):
    
        genotypes = input_library[i][input_library[i] != "-"]
        idx = np.array([genotype_to_idx[g] for g in genotypes])
    
        clone_genotypes.append(genotypes)
        clone_condition_k[i] = np.mean(condition_k[idx,:],axis=0)
        clone_base_delta_k[i] = np.mean(base_delta_k[idx])
    
    clone_genotypes = list(clone_genotypes)


In [ ]:
input_library = [
x, y = np.unique(input_library,return_counts=True)
input_library.shape


    

In [ ]:
base_k[0]

In [ ]:
df.to_csv("obs.csv")
genotype_df.to_csv("genotype.csv")
cfu_df.to_csv("cfu.csv")

In [ ]:
# libraries --> dictionary keying each library to a list of mutations in each clone
# 
# lib_phenotypes
# [{'clone': ['V30N'],
#    'ddG': array([2.0162, 1.8317, 1.8317, 2.0162, 2.0162, 1.8317, 1.8317, 2.0162,
#           1.8317, 1.8312, 1.8312, 1.8312, 1.831 , 1.8309, 1.8309, 1.8309,
#           0.    ]),
#    'fx_occupied': array([0.0007608 , 0.00103465, 0.00426162, 0.01198636, 0.02997511,
#           0.04687013, 0.05698534, 0.06055376]),
#    'fx_folded': array([1.        , 1.        , 0.99999996, 0.99999979, 0.99999928,
#           0.99999875, 0.99999842, 0.9999983 ]),
#    'obs': array([0.0007608 , 0.00103465, 0.00426162, 0.01198636, 0.02997509,
#           0.04687007, 0.05698525, 0.06055365]),
#    'base_growth_rate': np.float64(0.005445439477620463),
#    'sel_kan': array([3.98258294e-05, 5.41413358e-05, 2.22833015e-04, 6.26648000e-04,
#           1.56701933e-03, 2.45021391e-03, 2.97899038e-03, 3.16553093e-03]),
#    'kan_p0g': np.float64(5.4454394776204634e-08),
#    'kan_p1g': np.float64(0.004356351582096371),
#    'sel_pheS': array([0.00431658, 0.00430226, 0.00413357, 0.00372976, 0.00278939,
#           0.00190619, 0.00137742, 0.00119088]),
#    'pheS_p0g': np.float64(0.004356351582096371),
#    'pheS_p1g': np.float64(5.4454394776204634e-08)},

# input_library and all_genotypes
# input_library -> array([214305,   8702, 169508, ...,  81919, 281807,  70355],shape=(10000000,))

# all_genotypes -> list with full genotype information (like lib_phenotypes) indexed by input library.
# ln_pop_array : log population of each clone in initial library before selection (indexed by input_library)
# base_growth_rates : base growth rate of each clone (indexed by input_library)
# growth_rates : dictionary keyed to selection. values are N x num_iptg arrays where N is indexed by input_library


In [ ]:

all_genotypes = []
for lib in libraries:
    all_genotypes.extend(["/".join(g) for g in libraries[lib]])

genotype_df = build_genotype_df(all_genotypes)


In [ ]:
libraries[('1',)]